In [10]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight,tfidf_weight)
from tqdm import tqdm
import codecs
from tabulate import tabulate

In [2]:
retail_data = pd.read_json('data/cus_user_quality_date_0716.json') # This may take a couple minutes
# df2 = pd.read_csv('data/product_brand_type_color.csv') # This may take a couple minutes

In [3]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120442 entries, 0 to 120441
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMER_DIM_KEY   120442 non-null  int64  
 1   PRODUCT_DIM_KEY    120442 non-null  int64  
 2   QUANTITY           120441 non-null  float64
 3   PRODUCT_NAME       120435 non-null  object 
 4   COLOR              120442 non-null  object 
 5   PRODUCT_TYPE       120442 non-null  object 
 6   CATEGORY           120442 non-null  object 
 7   BRAND              120442 non-null  object 
 8   RESOLUTION_STATUS  120442 non-null  object 
 9   ORDER_DATE_KEY     120442 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 9.2+ MB


In [4]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]
cleaned_retail = cleaned_retail[cleaned_retail['PRODUCT_DIM_KEY'].isin(cleaned_retail[cleaned_retail['ORDER_DATE_KEY'] > 17533].PRODUCT_DIM_KEY)]
cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)
cleaned_retail = cleaned_retail[cleaned_retail.RESOLUTION_STATUS.isin(['COMPLETED', 'PENDING'])]
print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

Duplicated rows: 280


In [5]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51810 entries, 0 to 120440
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CUSTOMER_DIM_KEY   51810 non-null  int64  
 1   PRODUCT_DIM_KEY    51810 non-null  int64  
 2   QUANTITY           51810 non-null  float64
 3   PRODUCT_NAME       51810 non-null  object 
 4   COLOR              51810 non-null  object 
 5   PRODUCT_TYPE       51810 non-null  object 
 6   CATEGORY           51810 non-null  object 
 7   BRAND              51810 non-null  object 
 8   RESOLUTION_STATUS  51810 non-null  object 
 9   ORDER_DATE_KEY     51810 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.3+ MB


In [6]:
grouped_cleaned = cleaned_retail.groupby(['CUSTOMER_DIM_KEY']).sum().reset_index() # Group together
grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_customers = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive

customer_list = grouped_customers['CUSTOMER_DIM_KEY'].tolist()

G:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# group products by product_dim_key to sum the quantity
grouped_products = cleaned_retail.groupby(['PRODUCT_DIM_KEY','PRODUCT_NAME','PRODUCT_TYPE','BRAND'])['QUANTITY'].agg(sum)
grouped_products = grouped_products.reset_index().sort_values(['QUANTITY'],ascending=False)
grouped_products = grouped_products.query('QUANTITY > 0')
grouped_products

,PRODUCT_DIM_KEY,PRODUCT_NAME,PRODUCT_TYPE,BRAND,QUANTITY
8215,404102,Universal Glass,all purpose wine,Zalto,1783.0
8214,404101,Bordeaux Glass,red wine,Zalto,1109.0
8213,404100,Burgundy Glass,red wine,Zalto,850.0
8217,404104,Champagne Glass,champagne,Zalto,742.0
4433,304248,Milly Acrylic Tumbler Blue,"drinking glasses,bath tumblers",Mario Luca Giusti,721.0
...,...,...,...,...,...
9545,458004,Baby Bunny Keepsake Bowl,Unknown,Arthur Court,1.0
9541,457966,Bunny Luncheon Napkin Box,"napkins,storage containers",Arthur Court,1.0
9605,458751,Kitchen Timer by Michael Graves,Unknown,Alessi,1.0
9633,462256,"Kaarl Mixed Black Large Cocktail Spoons, Set O...",bar accessories,Blue Pheasant,1.0


In [8]:
customer_id = 31773
customer_purchases = cleaned_retail[cleaned_retail['CUSTOMER_DIM_KEY']==customer_id].groupby(['PRODUCT_DIM_KEY','PRODUCT_NAME','PRODUCT_TYPE','BRAND'])['QUANTITY'].agg(sum)
customer_purchases = customer_purchases.reset_index().sort_values(['QUANTITY'],ascending=False)
customer_purchases

,PRODUCT_DIM_KEY,PRODUCT_NAME,PRODUCT_TYPE,BRAND,QUANTITY
0,105507,"Festival Napkins 20 X 20 in, Four",napkins,Sferra,2.0


In [11]:
cus_top_list = pd.DataFrame(columns=['CUSTOMER_DIM_KEY', 'top_brand', 'top_type','brand_conf','type_conf']) 
for customer_id in tqdm(customer_list):
    customer_purchases = cleaned_retail[cleaned_retail['CUSTOMER_DIM_KEY']==customer_id].groupby(['PRODUCT_DIM_KEY','PRODUCT_NAME','PRODUCT_TYPE','BRAND'])['QUANTITY'].agg(sum)
    customer_purchases = customer_purchases.reset_index().sort_values(['QUANTITY'],ascending=False)
    customer_purchases

    # get brands of all items puchased by the customer
    customer_purchases_brand = cleaned_retail[cleaned_retail['CUSTOMER_DIM_KEY']==customer_id].groupby(['BRAND'])['QUANTITY'].agg(sum)
    customer_purchases_brand = customer_purchases_brand.reset_index().sort_values(['QUANTITY'],ascending=False)

    # The top buy item brand of the customer
    top_brand = customer_purchases_brand['BRAND'].iloc[0]
    brand_conf = customer_purchases_brand['QUANTITY'].iloc[0]/customer_purchases_brand['QUANTITY'].sum()

    # get types of all items puchased by the customer
    customer_purchases_type = cleaned_retail[cleaned_retail['CUSTOMER_DIM_KEY']==customer_id].groupby(['PRODUCT_TYPE'])['QUANTITY'].agg(sum)
    customer_purchases_type = customer_purchases_type.reset_index().sort_values(['QUANTITY'],ascending=False)

    # The top buy item type of the customer
    top_type = customer_purchases_type['PRODUCT_TYPE'].iloc[0]
    type_conf = customer_purchases_type['QUANTITY'].iloc[0]/customer_purchases_type['QUANTITY'].sum()

    cus = pd.DataFrame([[customer_id,top_brand, top_type, brand_conf, type_conf]]  , columns=['CUSTOMER_DIM_KEY', 'top_brand', 'top_type','brand_conf','type_conf']) 
    cus_top_list = pd.concat([cus_top_list,cus])


  0%|          | 132/27143 [00:01<05:10, 86.95it/s]G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
100%|██████████| 27143/27143 [06:55<00:00, 65.37it/s]


In [12]:
cus_top_list.iloc[0]

CUSTOMER_DIM_KEY      31773
top_brand            Sferra
top_type            napkins
brand_conf                1
type_conf                 1
Name: 0, dtype: object

In [13]:
# get type and brand of the item to find other items with the same type and brand
# type = grouped_products[grouped_products['PRODUCT_DIM_KEY']==item_id]['PRODUCT_TYPE'].iloc[0]
# brand = grouped_products[grouped_products['PRODUCT_DIM_KEY']==item_id]['BRAND'].iloc[0]


result = pd.DataFrame(columns=['CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY','PRODUCT_NAME','top_brand', 'top_type','brand_conf','type_conf']) 

for i in tqdm(range(len(cus_top_list))):
    # customer
    item = cus_top_list.iloc[i]
    # purchase history
    customer_purchases = cleaned_retail[cleaned_retail['CUSTOMER_DIM_KEY']==item.CUSTOMER_DIM_KEY].groupby(['PRODUCT_DIM_KEY','PRODUCT_NAME','PRODUCT_TYPE','BRAND'])['QUANTITY'].agg(sum)
    customer_purchases = customer_purchases.reset_index().sort_values(['QUANTITY'],ascending=False)

    # get 3 dataframes, both type and brand matched items, type matched items and brand matched items

    matched = grouped_products[grouped_products['PRODUCT_TYPE']==item.top_type][grouped_products['BRAND']==item.top_brand]
    type_matched = grouped_products[grouped_products['PRODUCT_TYPE']==item.top_type]
    brand_matched = grouped_products[grouped_products['BRAND']==item.top_brand]

    matched['recom_class']='both'
    type_matched['recom_class']='type'
    brand_matched['recom_class']='brand'

    # concat type and brand matched and sort by quantity
    sort_matched = pd.concat([type_matched,brand_matched]).sort_values(['QUANTITY'],ascending=False)

    recom = pd.concat([matched,sort_matched]).drop_duplicates().head(20)
    recom['CUSTOMER_DIM_KEY'] = cus_top_list.iloc[i].CUSTOMER_DIM_KEY
    recom['top_brand'] = cus_top_list.iloc[i].top_brand
    recom['top_type'] = cus_top_list.iloc[i].top_type
    recom['brand_conf'] = cus_top_list.iloc[i].brand_conf
    recom['type_conf'] = cus_top_list.iloc[i].type_conf
    
    #result.append(recom.values.tolist())
    result = pd.concat([result,recom])


  0%|          | 0/27143 [00:00<?, ?it/s]G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 27143/27143 [1:04:59<00:00,  6.96it/s]


In [17]:
result

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,PRODUCT_NAME,top_brand,top_type,brand_conf,type_conf,PRODUCT_TYPE,BRAND,QUANTITY,recom_class
3476,31773,194304,Juliet Four Napkins 22 X 22 In,Sferra,napkins,1.0,1.0,napkins,Sferra,222.0,both
136,31773,105537,"Festival Napkins 20 X 20 in, Four",Sferra,napkins,1.0,1.0,napkins,Sferra,185.0,both
107,31773,104018,Four Acanthus 22 X 22 Napkins,Sferra,napkins,1.0,1.0,napkins,Sferra,161.0,both
3477,31773,194308,Juliet Four Napkins 22 X 22 In,Sferra,napkins,1.0,1.0,napkins,Sferra,157.0,both
133,31773,105511,"Festival Napkins 20 X 20 in, Four",Sferra,napkins,1.0,1.0,napkins,Sferra,100.0,both
...,...,...,...,...,...,...,...,...,...,...,...
1102,83510,135741,Lastra White Condiment Bowl,Vietri,waste baskets,1.0,1.0,condiment holders,Vietri,48.0,brand
1682,83510,150412,Campagna Uccello Dinner Plate,Vietri,waste baskets,1.0,1.0,dinner plates,Vietri,47.0,brand
1284,83510,142186,Campagna Pesce Cereal/Soup Bowl,Vietri,waste baskets,1.0,1.0,cereal & soup bowls,Vietri,46.0,brand
529,83510,125173,Aladdin Clear Brilliant Three-Piece Child's Fl...,Vietri,waste baskets,1.0,1.0,flatware,Vietri,46.0,brand


In [119]:
#for row in customer_purchases.itertuples():
    #if row.PRODUCT_DIM_KEY in recom.PRODUCT_DIM_KEY.head(20).tolist():


SyntaxError: unexpected EOF while parsing (<ipython-input-119-d8125b37014f>, line 2)

In [19]:
result[result['CUSTOMER_DIM_KEY']==55974]

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,PRODUCT_NAME,top_brand,top_type,brand_conf,type_conf,PRODUCT_TYPE,BRAND,QUANTITY,recom_class
1457,55974,146802,Simply Anna Gold Bread & Butter Plate 6.25 in ...,Anna Weatherley,bread & butter plates,1.0,0.545455,bread & butter plates,Anna Weatherley,43.0,both
641,55974,126746,Bouquet of Flowers Bread & Butter Plate 6.25 i...,Anna Weatherley,bread & butter plates,1.0,0.545455,bread & butter plates,Anna Weatherley,7.0,both
1464,55974,147004,Simply Anna Polka Gold Bread & Butter Plate 6....,Anna Weatherley,bread & butter plates,1.0,0.545455,bread & butter plates,Anna Weatherley,3.0,both
7471,55974,392614,Vario White Bread & Butter Plate 7 1/2 In Tria...,Anna Weatherley,bread & butter plates,1.0,0.545455,bread & butter plates,Rosenthal,72.0,type
1454,55974,146790,Simply Anna Gold Dinner Plate 10.5 in Round,Anna Weatherley,bread & butter plates,1.0,0.545455,dinner plates,Anna Weatherley,72.0,brand
1262,55974,142067,Powder Blue Dinner Plate 10.5 in Round,Anna Weatherley,bread & butter plates,1.0,0.545455,dinner plates,Anna Weatherley,60.0,brand
605,55974,126625,Anna's Palette Aqua Green Dinner Plate 10.5 in...,Anna Weatherley,bread & butter plates,1.0,0.545455,dinner plates,Anna Weatherley,57.0,brand
562,55974,126130,"Blanc De Blanc Bread & Butter Plate 6.25""",Anna Weatherley,bread & butter plates,1.0,0.545455,bread & butter plates,Philippe Deshoulieres,49.0,type
197,55974,108370,Hemisphere White Bread & Butter Plate 6.25 in ...,Anna Weatherley,bread & butter plates,1.0,0.545455,bread & butter plates,J.L. Coquet,48.0,type
1266,55974,142073,Powder Blue Salad/ Dessert Plate 9 in Round,Anna Weatherley,bread & butter plates,1.0,0.545455,"dessert plates,salad & dessert plates",Anna Weatherley,47.0,brand


In [20]:
result[result['CUSTOMER_DIM_KEY']==74134]

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,PRODUCT_NAME,top_brand,top_type,brand_conf,type_conf,PRODUCT_TYPE,BRAND,QUANTITY,recom_class
3,74134,98182,Bubble Medium Dish 5 oz,J.L. Coquet,Unknown,0.875,0.25,Unknown,J.L. Coquet,39.0,both
2,74134,98181,Bubble Large Dish 7 oz,J.L. Coquet,Unknown,0.875,0.25,Unknown,J.L. Coquet,16.0,both
5,74134,98184,Bubble Mini Dish 2 oz,J.L. Coquet,Unknown,0.875,0.25,Unknown,J.L. Coquet,4.0,both
4,74134,98183,Bubble Small Dish 3.5 oz,J.L. Coquet,Unknown,0.875,0.25,Unknown,J.L. Coquet,4.0,both
11541,74134,493138,Labyrinthe 29-11 CM Engraved Plates,J.L. Coquet,Unknown,0.875,0.25,Unknown,J.L. Coquet,1.0,both
8670,74134,416774,Isabella Acrylic Goblet,J.L. Coquet,Unknown,0.875,0.25,Unknown,Juliska,144.0,type
13195,74134,510130,Bicos Frosted Black Goblet,J.L. Coquet,Unknown,0.875,0.25,Unknown,Vista Alegre,137.0,type
8232,74134,404120,Gravitas Omega,J.L. Coquet,Unknown,0.875,0.25,Unknown,Zalto,112.0,type
3682,74134,200928,Christofle Cream Cleaner with Sponge,J.L. Coquet,Unknown,0.875,0.25,Unknown,Christofle,97.0,type
7112,74134,368634,Assorted Giftware Butterfly Butter and Jam Spr...,J.L. Coquet,Unknown,0.875,0.25,Unknown,Couzon,94.0,type


In [21]:
result[result['CUSTOMER_DIM_KEY']==83076]

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,PRODUCT_NAME,top_brand,top_type,brand_conf,type_conf,PRODUCT_TYPE,BRAND,QUANTITY,recom_class
3,83076,98182,Bubble Medium Dish 5 oz,J.L. Coquet,Unknown,0.5,0.5,Unknown,J.L. Coquet,39.0,both
2,83076,98181,Bubble Large Dish 7 oz,J.L. Coquet,Unknown,0.5,0.5,Unknown,J.L. Coquet,16.0,both
5,83076,98184,Bubble Mini Dish 2 oz,J.L. Coquet,Unknown,0.5,0.5,Unknown,J.L. Coquet,4.0,both
4,83076,98183,Bubble Small Dish 3.5 oz,J.L. Coquet,Unknown,0.5,0.5,Unknown,J.L. Coquet,4.0,both
11541,83076,493138,Labyrinthe 29-11 CM Engraved Plates,J.L. Coquet,Unknown,0.5,0.5,Unknown,J.L. Coquet,1.0,both
8670,83076,416774,Isabella Acrylic Goblet,J.L. Coquet,Unknown,0.5,0.5,Unknown,Juliska,144.0,type
13195,83076,510130,Bicos Frosted Black Goblet,J.L. Coquet,Unknown,0.5,0.5,Unknown,Vista Alegre,137.0,type
8232,83076,404120,Gravitas Omega,J.L. Coquet,Unknown,0.5,0.5,Unknown,Zalto,112.0,type
3682,83076,200928,Christofle Cream Cleaner with Sponge,J.L. Coquet,Unknown,0.5,0.5,Unknown,Christofle,97.0,type
7112,83076,368634,Assorted Giftware Butterfly Butter and Jam Spr...,J.L. Coquet,Unknown,0.5,0.5,Unknown,Couzon,94.0,type


In [24]:
result[result['CUSTOMER_DIM_KEY']==47059]

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,PRODUCT_NAME,top_brand,top_type,brand_conf,type_conf,PRODUCT_TYPE,BRAND,QUANTITY,recom_class
8217,47059,404104,Champagne Glass,Zalto,champagne,0.4,0.2,champagne,Zalto,742.0,both
8215,47059,404102,Universal Glass,Zalto,champagne,0.4,0.2,all purpose wine,Zalto,1783.0,brand
8214,47059,404101,Bordeaux Glass,Zalto,champagne,0.4,0.2,red wine,Zalto,1109.0,brand
8213,47059,404100,Burgundy Glass,Zalto,champagne,0.4,0.2,red wine,Zalto,850.0,brand
8217,47059,404104,Champagne Glass,Zalto,champagne,0.4,0.2,champagne,Zalto,742.0,brand
8217,47059,404104,Champagne Glass,Zalto,champagne,0.4,0.2,champagne,Zalto,742.0,type
8233,47059,404121,Coupe/Tumbler,Zalto,champagne,0.4,0.2,drinking glasses,Zalto,284.0,brand
8216,47059,404103,White Wine Glass,Zalto,champagne,0.4,0.2,white wine,Zalto,271.0,brand
8221,47059,404108,Water Glass,Zalto,champagne,0.4,0.2,water goblets,Zalto,177.0,brand
4360,47059,304151,Dolce Vita Acrylic Flute Clear,Zalto,champagne,0.4,0.2,champagne,Mario Luca Giusti,169.0,type
